In [1]:
'''ResNet50 model for Keras.
# Reference:
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
Adapted from code contributed by BigMoyan.
'''
from __future__ import print_function

import numpy as np
import warnings

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions, _obtain_input_shape
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs
import os, sys

WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

class ResNet50():

    def __init__(self):
        self.model = self.getResNet50()

    def identity_block(self, input_tensor, kernel_size, filters, stage, block):
        """The identity block is the block that has no conv layer at shortcut.
        # Arguments
            input_tensor: input tensor
            kernel_size: defualt 3, the kernel size of middle conv layer at main path
            filters: list of integers, the filterss of 3 conv layer at main path
            stage: integer, current stage label, used for generating layer names
            block: 'a','b'..., current block label, used for generating layer names
        # Returns
            Output tensor for the block.
        """
        filters1, filters2, filters3 = filters
        if K.image_data_format() == 'channels_last':
            bn_axis = 3
        else:
            bn_axis = 1
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'

        x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
        x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
        x = Activation('relu')(x)

        x = Conv2D(filters2, kernel_size,
                   padding='same', name=conv_name_base + '2b')(x)
        x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
        x = Activation('relu')(x)

        x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
        x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

        x = layers.add([x, input_tensor])
        x = Activation('relu')(x)
        return x


    def conv_block(self, input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
        """conv_block is the block that has a conv layer at shortcut
        # Arguments
            input_tensor: input tensor
            kernel_size: defualt 3, the kernel size of middle conv layer at main path
            filters: list of integers, the filterss of 3 conv layer at main path
            stage: integer, current stage label, used for generating layer names
            block: 'a','b'..., current block label, used for generating layer names
        # Returns
            Output tensor for the block.
        Note that from stage 3, the first conv layer at main path is with strides=(2,2)
        And the shortcut should have strides=(2,2) as well
        """
        filters1, filters2, filters3 = filters
        if K.image_data_format() == 'channels_last':
            bn_axis = 3
        else:
            bn_axis = 1
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'

        x = Conv2D(filters1, (1, 1), strides=strides,
                   name=conv_name_base + '2a')(input_tensor)
        x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
        x = Activation('relu')(x)

        x = Conv2D(filters2, kernel_size, padding='same',
                   name=conv_name_base + '2b')(x)
        x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
        x = Activation('relu')(x)

        x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
        x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

        shortcut = Conv2D(filters3, (1, 1), strides=strides,
                          name=conv_name_base + '1')(input_tensor)
        shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

        x = layers.add([x, shortcut])
        x = Activation('relu')(x)
        return x

    def getResNet50(self, include_top=True, weights='imagenet',
                 input_tensor=None, input_shape=None,
                 pooling=None,
                 classes=1000):
        """Instantiates the ResNet50 architecture.
        Optionally loads weights pre-trained
        on ImageNet. Note that when using TensorFlow,
        for best performance you should set
        `image_data_format="channels_last"` in your Keras config
        at ~/.keras/keras.json.
        The model and the weights are compatible with both
        TensorFlow and Theano. The data format
        convention used by the model is the one
        specified in your Keras config file.
        # Arguments
            include_top: whether to include the fully-connected
                layer at the top of the network.
            weights: one of `None` (random initialization)
                or "imagenet" (pre-training on ImageNet).
            input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
                to use as image input for the model.
            input_shape: optional shape tuple, only to be specified
                if `include_top` is False (otherwise the input shape
                has to be `(224, 224, 3)` (with `channels_last` data format)
                or `(3, 224, 244)` (with `channels_first` data format).
                It should have exactly 3 inputs channels,
                and width and height should be no smaller than 197.
                E.g. `(200, 200, 3)` would be one valid value.
            pooling: Optional pooling mode for feature extraction
                when `include_top` is `False`.
                - `None` means that the output of the model will be
                    the 4D tensor output of the
                    last convolutional layer.
                - `avg` means that global average pooling
                    will be applied to the output of the
                    last convolutional layer, and thus
                    the output of the model will be a 2D tensor.
                - `max` means that global max pooling will
                    be applied.
            classes: optional number of classes to classify images
                into, only to be specified if `include_top` is True, and
                if no `weights` argument is specified.
        # Returns
            A Keras model instance.
        # Raises
            ValueError: in case of invalid argument for `weights`,
                or invalid input shape.
        """
        if weights not in {'imagenet', None}:
            raise ValueError('The `weights` argument should be either '
                             '`None` (random initialization) or `imagenet` '
                             '(pre-training on ImageNet).')

        if weights == 'imagenet' and include_top and classes != 1000:
            raise ValueError('If using `weights` as imagenet with `include_top`'
                             ' as true, `classes` should be 1000')

        # Determine proper input shape
        input_shape = _obtain_input_shape(input_shape,
                                          default_size=224,
                                          min_size=197,
                                          data_format=K.image_data_format(),
                                          include_top=include_top)

        if input_tensor is None:
            img_input = Input(shape=input_shape)
        else:
            if not K.is_keras_tensor(input_tensor):
                img_input = Input(tensor=input_tensor, shape=input_shape)
            else:
                img_input = input_tensor
        if K.image_data_format() == 'channels_last':
            bn_axis = 3
        else:
            bn_axis = 1

        x = ZeroPadding2D((3, 3))(img_input)
        x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
        x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        x = self.conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
        x = self.identity_block(x, 3, [64, 64, 256], stage=2, block='b')
        x = self.identity_block(x, 3, [64, 64, 256], stage=2, block='c')

        x = self.conv_block(x, 3, [128, 128, 512], stage=3, block='a')
        x = self.identity_block(x, 3, [128, 128, 512], stage=3, block='b')
        x = self.identity_block(x, 3, [128, 128, 512], stage=3, block='c')
        x = self.identity_block(x, 3, [128, 128, 512], stage=3, block='d')

        x = self.conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
        x = self.identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
        x = self.identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
        x = self.identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
        x = self.identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
        x = self.identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

        x = self.conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
        x = self.identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
        x = self.identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

        x = AveragePooling2D((7, 7), name='avg_pool')(x)

        if include_top:
            x = Flatten()(x)
            x = Dense(classes, activation='softmax', name='fc1000')(x)
        else:
            if pooling == 'avg':
                x = GlobalAveragePooling2D()(x)
            elif pooling == 'max':
                x = GlobalMaxPooling2D()(x)

        # Ensure that the model takes into account
        # any potential predecessors of `input_tensor`.
        if input_tensor is not None:
            inputs = get_source_inputs(input_tensor)
        else:
            inputs = img_input
        # Create model.
        model = Model(inputs, x, name='resnet50')

        # load weights
        if weights == 'imagenet':
            if include_top:
                weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                        WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
            else:
                weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                        WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='a268eb855778b3df3c7506639542a6af')
            model.load_weights(weights_path)
            if K.backend() == 'theano':
                layer_utils.convert_all_kernels_in_model(model)

            if K.image_data_format() == 'channels_first':
                if include_top:
                    maxpool = model.get_layer(name='avg_pool')
                    shape = maxpool.output_shape[1:]
                    dense = model.get_layer(name='fc1000')
                    layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

                if K.backend() == 'tensorflow':
                    warnings.warn('You are using the TensorFlow backend, yet you '
                                  'are using the Theano '
                                  'image data format convention '
                                  '(`image_data_format="channels_first"`). '
                                  'For best performance, set '
                                  '`image_data_format="channels_last"` in '
                                  'your Keras config '
                                  'at ~/.keras/keras.json.')
        return model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
resnet = ResNet50()

In [3]:
resnet.model.layers.pop()

In [4]:
new_softmax = Dense(3, activation='softmax', name='new_softmax')

In [5]:
inp = resnet.model.input

In [6]:
out = new_softmax(resnet.model.layers[-1].output)

In [7]:
model2 = Model(inp,out)

In [8]:
from keras.optimizers import Adam

In [9]:
model2.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=16, class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(224,224),
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)


In [11]:
train_path = '/home/ubuntu/kaggle_cervical_cancer_code/data/segmented/train'

In [17]:
train_batches = get_batches(train_path, batch_size=128)

Found 6166 images belonging to 3 classes.


In [47]:
val_path = '/home/ubuntu/kaggle_cervical_cancer_code/data/segmented/valid'

In [19]:
val_batches = get_batches(val_path, batch_size=4)

Found 1461 images belonging to 3 classes.


In [20]:
for layer in model2.layers:
    layer.trainable = False

In [21]:
model2.layers[-1].trainable=True

In [22]:
model2.fit_generator(train_batches, 
                     steps_per_epoch=50, validation_data=val_batches, validation_steps=12)

Epoch 1/1


MemoryError: Error allocating 102760448 bytes of device memory (out of memory).
Apply node that caused the error: GpuElemwise{add,no_inplace}(GpuDimShuffle{0,2,3,1}.0, GpuDimShuffle{x,x,x,0}.0)
Toposort index: 4316
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, (True, True, True, False))]
Inputs shapes: [(128, 14, 14, 1024), (1, 1, 1, 1024)]
Inputs strides: [(200704, 14, 1, 196), (0, 0, 0, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuShape(GpuElemwise{add,no_inplace}.0), GpuReshape{2}(GpuElemwise{add,no_inplace}.0, MakeVector{dtype='int64'}.0), GpuElemwise{Composite{(((i0 - i1) * i2) + i3)},no_inplace}(GpuElemwise{add,no_inplace}.0, GpuDimShuffle{x,x,x,0}.0, GpuElemwise{Composite{(i0 / sqrt((i1 + i2)))},no_inplace}.0, GpuDimShuffle{x,x,x,0}.0), GpuElemwise{sub,no_inplace}(GpuElemwise{add,no_inplace}.0, GpuElemwise{Composite{(((i0 / i1) / i2) / i3)}}[(0, 0)].0), GpuElemwise{Composite{(((i0 - i1) * i2 * i3) + i4)},no_inplace}(GpuElemwise{add,

In [54]:
precompute_train_batches = get_batches(train_path, shuffle=False, batch_size=2, class_mode=None)

Found 6166 images belonging to 3 classes.


In [34]:
6166/2

3083

In [55]:
precompute_train_values = resnet.model.predict_generator(precompute_train_batches, steps=3083)

In [36]:
precompute_train_values.shape

(6166, 1000)

In [37]:
keras_tensor = K.variable(value=precompute_train_values)

In [38]:
input_out = Input(shape=(6166,1000), batch_shape=(128,1000), name="output_conv_layers", tensor=keras_tensor)

In [39]:
new_softmax = Dense(3, activation='softmax', name='new_softmax')(input_out)

In [90]:
'''
See how you can now start to train new_softmax. This should give you the opportunity to ONLY train the 
softmax layer, instead of propagating computing back-n-forth the entire network. Precisely the point
of fine-tuning.

''' 


'\nSee how you can now start to train new_softmax. This should give you the opportunity to ONLY train the \nsoftmax layer, instead of propagating computing back-n-forth the entire network. Precisely the point\nof fine-tuning.\n\n'

In [40]:
model3 = Model(inputs=input_out, outputs=new_softmax)

In [41]:
model3.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
from keras.utils.np_utils import to_categorical
def onehot(x):
    return to_categorical(x)

In [43]:
precompute_train_batches.classes

array([0, 0, 0, ..., 2, 2, 2], dtype=int32)

In [44]:
precompute_train_labels = onehot(precompute_train_batches.classes)

In [45]:
precompute_train_labels.shape

(6166, 3)

In [49]:
precompute_valid_batches = get_batches(val_path, shuffle=False, batch_size=1, class_mode=None)

Found 1461 images belonging to 3 classes.


In [52]:
precompute_valid_values = resnet.model.predict_generator(precompute_valid_batches, steps=1461)

In [53]:
precompute_valid_labels = onehot(precompute_valid_batches.classes)

In [ ]:
precom

In [56]:
model3.fit(precompute_train_values, precompute_train_labels, nb_epoch=2,
                       batch_size= 128, validation_data=(precompute_valid_values, precompute_valid_labels))

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 6166 samples, validate on 1461 samples
Epoch 1/2


ValueError: GpuElemwise. Input dimension mis-match. Input 1 (indices start at 0) has shape[0] == 6166, but the output's size on that axis is 128.
Apply node that caused the error: GpuElemwise{Composite{(i0 * log(i1))},no_inplace}(GpuFromHost.0, GpuElemwise{Clip}[(0, 0)].0)
Toposort index: 45
Inputs types: [CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]
Inputs shapes: [(128, 3), (6166, 3)]
Inputs strides: [(3, 1), (3, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuCAReduce{add}{0,1}(GpuElemwise{Composite{(i0 * log(i1))},no_inplace}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.